# Capstone Project-The Battle of Neighborhoods

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

# Introduction: Business Problem


   Suppose that a new constructor in New York City, looking for an existing commercial and residential building, is eager to find a building that is met with two important criterials. **Quiet and comfortable with a park nearby** and **Higher education such as colleges and museums is reachable**.
    
   To simply the problem, this analysis is only focus surroundings that will influence the decision without considering other internal and external defining factors. Also, below constraints and conditions must be met to limit the scope of this analysis:
    a. Regardless of building price
    b. Only applying Foursquare location data

# Data
Based on definition of our problem, factors that will influence our decission are:
* number of existing parks in the neighborhood near any building
* number of and distance to museums in the neighborhood, if any
* distance of neighborhood from nearby colleges

We decided to use regularly spaced grid of locations, centered around borough, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* Geospatial and building data is obtained from a previous Kaggle competition for "NYC Property Sales" between Sep 2016 to Sep 2017
* number of colleges and mesuems location in every neighborhood will be obtained using **Foursquare API**

**"NYC Property Sales" between Sep 2016 to Sep 2017 dataset** contains the location, address, type, sale price, and sale date of building units sold. 

A reference on the trickier fields:

* **BOROUGH**: A digit code for the borough the property is located in; in order these are Manhattan (1), Bronx (2), Brooklyn (3), Queens (4), and Staten Island (5).
* **BLOCK; LOT**: The combination of borough, block, and lot forms a unique key for property in New York City. Commonly called a BBL.
* **BUILDING CLASS AT PRESENT** and **BUILDING CLASS AT TIME OF SALE**: The type of building at various points in time. 

In [1]:
import pandas as pd
import numpy as np

In [228]:
nyc_df = pd.read_csv('nyc-rolling-sales.csv')
nyc_df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


Learn some basic information about the New York Building data

In [229]:
# see columns and shape of the dataset
print(nyc_df.shape)
nyc_df.columns

(84548, 22)


Index(['Unnamed: 0', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [230]:
nyc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
Unnamed: 0                        84548 non-null int64
BOROUGH                           84548 non-null int64
NEIGHBORHOOD                      84548 non-null object
BUILDING CLASS CATEGORY           84548 non-null object
TAX CLASS AT PRESENT              84548 non-null object
BLOCK                             84548 non-null int64
LOT                               84548 non-null int64
EASE-MENT                         84548 non-null object
BUILDING CLASS AT PRESENT         84548 non-null object
ADDRESS                           84548 non-null object
APARTMENT NUMBER                  84548 non-null object
ZIP CODE                          84548 non-null int64
RESIDENTIAL UNITS                 84548 non-null int64
COMMERCIAL UNITS                  84548 non-null int64
TOTAL UNITS                       84548 non-null int64
LAND SQUARE FEET                  84548 non-null object
GRO

In [231]:
nyc_df.describe()

,Unnamed: 0,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
count,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000
mean,10344.359878,2.998758,4237.218976,376.224015,10731.991614,2.025264,0.193559,2.249184,1789.322976,1.657485
std,7151.779436,1.289790,3568.263407,658.136814,1290.879147,16.721037,8.713183,18.972584,537.344993,0.819341
min,4.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4231.000000,2.000000,1322.750000,22.000000,10305.000000,0.000000,0.000000,1.000000,1920.000000,1.000000
50%,8942.000000,3.000000,3311.000000,50.000000,11209.000000,1.000000,0.000000,1.000000,1940.000000,2.000000
75%,15987.250000,4.000000,6281.000000,1001.000000,11357.000000,2.000000,0.000000,2.000000,1965.000000,2.000000
max,26739.000000,5.000000,16322.000000,9106.000000,11694.000000,1844.000000,2261.000000,2261.000000,2017.000000,4.000000


### Clean dataset

In [232]:
# Renaming BOROUGHS
nyc_df['BOROUGH'][nyc_df['BOROUGH'] == 1] = 'Manhattan'
nyc_df['BOROUGH'][nyc_df['BOROUGH'] == 2] = 'Bronx'
nyc_df['BOROUGH'][nyc_df['BOROUGH'] == 3] = 'Brooklyn'
nyc_df['BOROUGH'][nyc_df['BOROUGH'] == 4] = 'Queens'
nyc_df['BOROUGH'][nyc_df['BOROUGH'] == 5] = 'Staten Island'

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [226]:
#see all columns in the dataset
pd.set_option('display.max_columns',999)
nyc_df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


Drop some rows with null data and drop an empty column "EASE-MENT" 

In [233]:
nyc_df.dropna(axis=0, how='any', inplace=True)
# Unnamed: 0 is an artifact from the data load and can be deleted
del nyc_df['Unnamed: 0']
nyc_df.shape

(84548, 21)

Check duplicates and delete those rows

In [235]:
sum(nyc_df.duplicated(nyc_df.columns))

765

In [237]:
#Delete the duplicates and check
nyc_df = nyc_df.drop_duplicates(nyc_df.columns, keep='last')
sum(nyc_df.duplicated(nyc_df.columns))

0

Create a new dataframe called nyc_borough contains BOROUGH, NEIGHBORHOOD, ZIP CODE and BUILDING CLASS CATEGORY

In [238]:
nyc_borough=nyc_df[['BOROUGH', 'NEIGHBORHOOD', 'ZIP CODE', 'BUILDING CLASS CATEGORY']]
nyc_borough.head()

,BOROUGH,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS CATEGORY
0,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS
1,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS
2,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS
3,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS
4,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS


In [250]:
borough_grouped=nyc_borough.groupby(['BOROUGH','NEIGHBORHOOD']).count()
borough_grouped

ZIP CODE  BUILDING CLASS CATEGORY
BOROUGH       NEIGHBORHOOD                                                
Bronx         BATHGATE                         68                       68
              BAYCHESTER                      436                      436
              BEDFORD PARK/NORWOOD            203                      203
              BELMONT                          89                       89
              BRONX PARK                        1                        1
              BRONXDALE                       326                      326
              CASTLE HILL/UNIONPORT           187                      187
              CITY ISLAND                      91                       91
              CITY ISLAND-PELHAM STRIP          1                        1
              CO-OP CITY                        5                        5
              COUNTRY CLUB                    165                      165
              CROTONA PARK                     62                       62
              EAST RIVER                        1                        1
              EAST TREMONT                    145                      145
              FIELDSTON                        46                       46
              FORDHAM                          78                       78
              HIGHBRIDGE/MORRIS HEIGHTS       129                      129
              HUNTS POINT                      74                       74
              KINGSBRIDGE HTS/UNIV HTS        130                      130
              KINGSBRIDGE/JEROME PARK         208                      208
              MELROSE/CONCOURSE               242                      242
              MORRIS PARK/VAN NEST            276                      276
              MORRISANIA/LONGWOOD             355                      355
              MOTT HAVEN/PORT MORRIS          138                      138
              MOUNT HOPE/MOUNT EDEN            88                       88
              PARKCHESTER                     450                      450
              PELHAM BAY                        1                        1
              PELHAM GARDENS                   80                       80
              PELHAM PARKWAY NORTH            145                      145
              PELHAM PARKWAY SOUTH            133                      133
...                                           ...                      ...
Staten Island NEW DORP                        178                      178
              NEW DORP-BEACH                  117                      117
              NEW DORP-HEIGHTS                105                      105
              NEW SPRINGVILLE                 537                      537
              OAKWOOD                          40                       40
              OAKWOOD-BEACH                    65                       65
              PLEASANT PLAINS                  66                       66
              PORT IVORY                       68                       68
              PORT RICHMOND                   288                      288
              PRINCES BAY                     144                      144
              RICHMONDTOWN                     88                       88
              RICHMONDTOWN-LIGHTHS HILL        16                       16
              ROSEBANK                        198                      198
              ROSSVILLE                       182                      182
              ROSSVILLE-CHARLESTON             62                       62
              ROSSVILLE-PORT MOBIL              6                        6
              ROSSVILLE-RICHMOND VALLEY        16                       16
              SILVER LAKE                      62                       62
              SOUTH BEACH                     230                      230
              STAPLETON                       116                      116
              STAPLETON-CLIFTON                65                 

Import postcode coordinates and clean.

New York City coordinates is 40.7128° N, 74.0060° W

In [263]:
postcode_df=pd.read_csv('postcode.csv')
postcode_df.head()

,Place,Code,lat,lon,PostCode,Admin1,Admin2,Admin3
0,1,New York,40.748,-73.997,10001,United States,New York,New York
1,2,New York,40.715,-73.988,10002,United States,New York,New York
2,3,New York,40.731,-73.989,10003,United States,New York,New York
3,4,New York,40.714,-74.006,10004,United States,New York,New York
4,5,New York,40.706,-74.008,10005,United States,New York,New York


In [189]:
postcode_df.columns

Index(['Place', 'Code', 'lat', 'lon', 'PostCode', 'Admin1', 'Admin2',
       'Admin3'],
      dtype='object')

In [264]:
#drop some columns irrelevent to this analysis
postcode_df=postcode_df.drop(columns=['Place', 'Admin1','Admin2', 'Admin3', 'Code'])

In [265]:
postcode_df=postcode_df[['PostCode', 'lat', 'lon']]
postcode_df.head()

,PostCode,lat,lon
0,10001,40.748,-73.997
1,10002,40.715,-73.988
2,10003,40.731,-73.989
3,10004,40.714,-74.006
4,10005,40.706,-74.008


Merge two dataframes, postcode_df and nyc_borough, on zip code to create a new dataframe called nyc_merged

In [301]:
nyc_merged=nyc_borough.join(postcode_df.set_index('PostCode'), on='ZIP CODE')
nyc_merged.head()

,BOROUGH,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS CATEGORY,lat,lon
0,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS,40.726,-73.98
1,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS,40.726,-73.98
2,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS,40.726,-73.98
3,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS,40.726,-73.98
4,Manhattan,ALPHABET CITY,10009,07 RENTALS - WALKUP APARTMENTS,40.726,-73.98


In [305]:
nyc_merged.dropna(axis=0, how='any')
nyc_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83783 entries, 0 to 84547
Data columns (total 6 columns):
BOROUGH                    83783 non-null object
NEIGHBORHOOD               83783 non-null object
ZIP CODE                   83783 non-null int64
BUILDING CLASS CATEGORY    83783 non-null object
lat                        32341 non-null float64
lon                        32341 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 4.5+ MB


### Explore and cluster the neighborhoods in New York City

In [271]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nyc_merged['BOROUGH'].unique()),
        nyc_merged.shape[0]
    )
)

The dataframe has 5 boroughs and 83783 neighborhoods.


In [272]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

# All requested packages already installed.

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [273]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [274]:
from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    try:
        return geopy.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

### Create a map of New York City with neighborhoods superimposed on it

In [276]:
# get new york coordinates
address = 'NEW YORK, US'

geolocator = Nominatim(user_agent="NY_explorer")
location = geolocator.geocode(address, timeout=10)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7127281, -74.0060152.


In [278]:
# create map of Toronto using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nyc_merged['lat'], nyc_merged['lon'], nyc_merged['BOROUGH'], nyc_merged['NEIGHBORHOOD']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

ValueError: Location values cannot contain NaNs, got:
[nan, nan]